In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd


In [5]:
df = pd.read_excel('justwatch.xlsx')

In [6]:
df.tail(2)

,Unnamed: 0,title,original_title,year,season_episode,runtime,genre,age_rating,Production country
1088,1088,대송궁사,大宋宫词,2021,1,45min,드라마,15+,China
1089,1089,"엘리트들, 못다 한 이야기: 나디아 구스만",Elite Histórias Breves: Nadia Guzmán,2021,1,11min,"드라마, 로맨스",NaN,스페인


In [96]:
# 크롬 드라이버 설정 및 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/?ref_=nv_home')

In [97]:
# 크롬 드라이버 설정 및 페이지 열기
def driver_chrome(url):
    driver = webdriver.Chrome()
    driver.get(url)

# 메인 페이지 검색 
def search_drama(title):
    m_search_box  = driver.find_element(By.CSS_SELECTOR, '#suggestion-search')
    m_search_box.send_keys(title)

# 맨 처음에 뜨는 드라마 선택
def first_drama_select():
    first_drama_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#react-autowhatever-navSuggestionSearch--item-0 > a'))
    )
    first_drama_result.click()

# 첫 번째 페이지에서 디테일 데이터 수집 
def first_page_data():

    # 페이지 소스 가져오기
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    #디테일 정보 가져오기 
    title = soup.select_one('div.sc-491663c0-3.bdjVSf > div.sc-b7c53eda-0.dUpRPQ > h1 > span').text.strip() if soup.select_one('div.sc-491663c0-3.bdjVSf > div.sc-b7c53eda-0.dUpRPQ > h1 > span') else 'N/A'
    original_title_o = soup.select_one('div.sc-b7c53eda-0.dUpRPQ > div').text.strip() if soup.select_one('div.sc-b7c53eda-0.dUpRPQ > div') else 'N/A'
    original_title = original_title_o.replace('Original title: ', '')
    year = soup.select_one('div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2) > a').text.strip() if soup.select_one('div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2) > a') else 'N/A'


    

In [118]:
# 크롬 드라이버 설정 및 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/?ref_=nv_home')

#justwach에서 크롤링로 얻은 원제를 검색해 데이터 수집 
# for ot in df['original_title']

first_pages_detail_data = []
# 메인 페이지 검색 
m_search_box  = driver.find_element(By.CSS_SELECTOR, '#suggestion-search')
m_search_box.send_keys('La Casa de Papel')


try:
    # 맨 처음에 뜨는 드라마 선택
    first_drama_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#react-autowhatever-navSuggestionSearch--item-0 > a'))
    )
    first_drama_result.click()


    # 첫 번째 페이지에서 디테일 데이터 수집 
    # 페이지 소스 가져오기
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    #디테일 정보 가져오기 
    title = soup.select_one('div.sc-491663c0-3.bdjVSf > div.sc-b7c53eda-0.dUpRPQ > h1 > span').text.strip() if soup.select_one('div.sc-491663c0-3.bdjVSf > div.sc-b7c53eda-0.dUpRPQ > h1 > span') else 'N/A'
    original_title_o = soup.select_one('div.sc-b7c53eda-0.dUpRPQ > div').text.strip() if soup.select_one('div.sc-b7c53eda-0.dUpRPQ > div') else 'N/A'
    original_title = original_title_o.replace('Original title: ', '')
    year = soup.select_one('div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2) > a').text.strip() if soup.select_one('div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2) > a') else 'N/A'
    total_avg_star_rating = soup.select_one('div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK').text.strip() if soup.select_one('div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK') else 'N/A'
    total_number_of_stars = soup.select_one('div.sc-bde20123-0.dLwiNw > div.sc-bde20123-3.gPVQxL').text.strip() if soup.select_one('div.sc-bde20123-0.dLwiNw > div.sc-bde20123-3.gPVQxL') else 'N/A'
    add_to_watchlist = soup.select_one('button.ipc-split-button__btn > div > div.sc-b23676b3-3.bRgISf').text.strip() if soup.select_one('button.ipc-split-button__btn > div > div.sc-b23676b3-3.bRgISf') else 'N/A'
    add_to_watchlist_num = add_to_watchlist.split('명의')[0]
    
    drama_details = {
        'title': title,
        'original_title': original_title,
        'year': year,
        'total_avg_star_rating': total_avg_star_rating,
        'total_number_of_stars': total_number_of_stars,
        'add_to_watchlist_num': add_to_watchlist_num
    }

    first_pages_detail_data.append(drama_details)

    print(first_pages_detail_data)
except Exception as e:
    print(e)


[{'title': 'La casa de papel', 'original_title': 'N/A', 'year': '2017–2021', 'total_avg_star_rating': '8.2', 'total_number_of_stars': '536K', 'add_to_watchlist_num': '45만'}]


---
### 시즌별 에피소드 디테일 

In [119]:
seasons_data = []

# 시즌 셀렉트 박스 찾기
try:
    season_select_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'browse-episodes-season'))
    )
except Exception as e:
    print("No seasons found or element not loaded properly")
    

# 'SEE_ALL' 옵션 찾기 및 클릭
for option in season_select_box.find_elements(By.TAG_NAME,'option'):
    season_num = option.get_attribute('value')

    if season_num == 'SEE_ALL':
        option.click() 
        
page_source_season = driver.page_source
soup_season = BeautifulSoup(page_source_season, 'html.parser')  

In [120]:
# 시즌 페이지에서 에피소드별 디테일 

#시즌 선택 
season_num_ul = soup_season.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-56501f3b-0.iZwhod > div.ipc-tabs.ipc-tabs--base.ipc-tabs--align-left.ipc-tabs--display-chip.ipc-tabs--inherit > ul')

#에피소드 정보 크롤링
episodes_data = []

# 시즌 별 클릭 
# for season in season_select_box.select(By.CSS_SELECTOR, 'div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-56501f3b-0.iZwhod > div.ipc-tabs.ipc-tabs--base.ipc-tabs--align-left.ipc-tabs--display-chip.ipc-tabs--inherit > ul'):
#     print(season)
#     season_button = season.find_element(By.TAG_NAME, 'a')
#     season_button.click()
#     time.sleep(1)

episodes = soup_season.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW > article')


for episode in episodes:
    episode_detail = episode.select_one('div.sc-ccd6e31b-5.cXcoHr > h4 > div > a > div').text.strip() if episode.select_one('div.sc-ccd6e31b-5.cXcoHr > h4 > div > a > div') else 'N/A'
    
    if episode_detail != 'N/A':
        season = episode_detail.split('.')[0]
        episode_num = episode_detail.split('.')[1].split(' ∙ ')[0]
        episode_name = episode_detail.split('.')[1].split(' ∙ ')[1]
        
        episode_data = {
            'season': season,
            'episode': episode_num,
            'episode_name': episode_name
        }
        episodes_data.append(episode_data)

episodes_data



[{'season': 'S1', 'episode': 'E1', 'episode_name': 'Efectuar lo acordado'},
 {'season': 'S1', 'episode': 'E2', 'episode_name': 'Imprudencias letales'},
 {'season': 'S1', 'episode': 'E3', 'episode_name': 'Errar al disparar'},
 {'season': 'S1', 'episode': 'E4', 'episode_name': 'Caballo de Troya'},
 {'season': 'S1', 'episode': 'E5', 'episode_name': 'El día de la marmota'},
 {'season': 'S1', 'episode': 'E6', 'episode_name': 'La cálida Guerra Fría'},
 {'season': 'S1',
  'episode': 'E7',
  'episode_name': 'Refrigerada inestabilidad'},
 {'season': 'S1', 'episode': 'E8', 'episode_name': 'Tú lo has buscado'},
 {'season': 'S1',
  'episode': 'E9',
  'episode_name': 'El que la sigue la consigue'}]

In [87]:
episode_detail = soup_season.select_one('div.sc-ccd6e31b-5.cXcoHr > h4 > div > a > div').text.strip() if soup_season.select_one('div.sc-ccd6e31b-5.cXcoHr > h4 > div > a > div') else 'N/A'
season = episode_detail.split('.')[0]
episode = episode_detail.split('.')[1].split(' ∙ ')[0]
episode_name = episode_detail.split('.')[1].split(' ∙ ')[1]

['E1', 'Mugunghwa kkoch-i pideon nal']

In [ ]:
#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW > article:nth-child(1) > div > div > div.sc-ccd6e31b-4.eMYVLm > div.sc-ccd6e31b-5.cXcoHr > h4 > div > a > div